In [ ]:
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers diffusers requests beautifulsoup4 praw flask flask-cors pyngrok -q


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("enter grok auth token")

In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import HuggingFacePipeline
from diffusers import StableDiffusionPipeline
import praw
import warnings
warnings.filterwarnings("ignore")
import time
from datetime import datetime, timedelta
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
import base64
from io import BytesIO



In [ ]:
# ==============================================================================
# 2. Authenticate ngrok and Load AI Models
# ==============================================================================
print("1. Authenticating ngrok...")
# You must paste your ngrok authtoken here to run the web application.
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
# ngrok.set_auth_token("YOUR_AUTHTOKEN_HERE")

print("2. Loading AI models... This may take a few minutes.")

llm_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True
)
llm = HuggingFacePipeline(pipeline=pipe)

sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
stable_diffusion_pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base",
    torch_dtype=torch.float16
).to("cuda")

print("✅ All models loaded successfully!")

In [ ]:
# ==============================================================================
# 3. Core Functions & Social Media API Integration
# ==============================================================================

# --- Social Media API Credentials (Update with your own keys) ---
# IMPORTANT: DO NOT share these with anyone. This is for your personal use.

# Twitter API (You may need to uncomment and install the library)
!pip install tweepy -q
TWITTER_API_KEY = "***"
TWITTER_API_SECRET = "***"
TWITTER_ACCESS_TOKEN = "***"
TWITTER_ACCESS_TOKEN_SECRET = "***"

# Facebook API (You may need to uncomment and install the library)
!pip install facebook-sdk -q
FACEBOOK_ACCESS_TOKEN = "***"
# Reddit API (You may need to uncomment and install the library)
!pip install praw -q
REDDIT_CLIENT_ID = "***"
REDDIT_CLIENT_SECRET = "***"
REDDIT_USERNAME = "***"
REDDIT_PASSWORD = "****"
REDDIT_USER_AGENT = "***"

# LinkedIn API (NOTE: This API is highly restricted for posting. You need to apply to their developer program.)
LINKEDIN_ACCESS_TOKEN = "***"
LINKEDIN_ORGANIZATION_ID = "***"

# Global function to store your scheduled posts in memory
# In a real-world app, you would use a database.
scheduled_posts_db = []
performance_metrics_db = {} # In-memory database for performance metrics

# --- Content Generation & Scraping Functions (Updated for multi-format) ---
def get_industry_trends(keyword):
    """Scrapes Google News for recent headlines related to a keyword."""
    url = f"https://news.google.com/search?q={keyword}&hl=en-US&gl=US&ceid=US:en"
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        headlines = [h.get_text() for h in soup.find_all('a', class_='JtKR7b')]
        return random.choice(headlines) if headlines else f"The latest advancements in {keyword}"
    except Exception:
        return f"The latest advancements in {keyword}"

def get_reddit_trends(subreddit_name):
    """Scrapes a public subreddit for popular post titles."""
    url = f"https://www.reddit.com/r/{subreddit_name}/hot.json"
    headers = {'User-Agent': 'MyBot/0.1'}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        titles = [post['data']['title'] for post in data['data']['children']]
        return random.choice(titles) if titles else f"The latest trends in r/{subreddit_name}"
    except Exception as e:
        return f"The latest trends in r/{subreddit_name}"

def find_subreddit(keyword):
    """Automatically finds a relevant subreddit based on a search keyword."""
    search_query = f"popular subreddits for {keyword}"
    url = f"https://www.google.com/search?q={search_query}"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a', href=True):
            if "reddit.com/r/" in link['href']:
                subreddit = link['href'].split('/r/')[1].split('/')[0]
                return subreddit
    except Exception:
        pass
    return "technology"

def generate_content(profile, trend, post_type, language="English"):
    """Generates content based on post type."""
    if post_type == "text":
        template = """
        You are an expert LinkedIn content creator. Your task is to write two professional and engaging LinkedIn posts for a user based on their profile and a recent industry trend.
        The posts should be in {language} and align with the user's brand voice.

        User Profile:
        - Name: {name}
        - Role: {role} at {company}
        - Skills: {skills}
        - Interests: {interests}
        - Brand Voice: {brand_voice}

        Industry Trend to Discuss: {trend}

        Instructions:
        1. Write two different posts (Post A and Post B).
        2. Each post should be 2-3 paragraphs long and start with a hook.
        3. Conclude each post with a question to encourage discussion.
        4. Include 5-7 relevant hashtags at the end of each post.
        5. The tone should match the user's specified brand voice.
        6. Do not include a signature or a closing.
        7. Separate the two posts clearly with "---".

        Generated LinkedIn Posts:
        Post A:
        """
        prompt = PromptTemplate(
            input_variables=["name", "role", "company", "skills", "interests", "brand_voice", "trend", "language"],
            template=template,
        )
        llm_chain = LLMChain(prompt=prompt, llm=llm)

        response = llm_chain.run(
            name=profile["name"],
            role=profile["role"],
            company=profile["company"],
            skills=", ".join(profile["skills"]),
            interests=", ".join(profile["interests"]),
            brand_voice=profile["brand_voice"],
            trend=trend,
            language=language
        )
        return [p.strip() for p in response.split("---")]

    elif post_type == "carousel":
        template = """
        You are an expert LinkedIn content creator. Your task is to create a 5-slide carousel post based on a user's profile and an industry trend. The posts should be in {language} and align with the user's brand voice.

        User Profile:
        - Name: {name}
        - Role: {role} at {company}
        - Skills: {skills}
        - Interests: {interests}
        - Brand Voice: {brand_voice}

        Industry Trend to Discuss: {trend}

        Instructions:
        1. Create a compelling title for the carousel.
        2. Create 5 concise slides, each with a title and a body of text.
        3. Slide 1 should be an attention-grabbing hook.
        4. Slides 2-4 should contain key insights or data points.
        5. Slide 5 should be a clear call-to-action (CTA) or a question.
        6. Format the output as a JSON object with keys "title", "slides", and "hashtags". Each slide should have "slide_title" and "slide_body".
        7. Include 5-7 relevant hashtags at the end.

        Example JSON output:
        {{
          "title": "5 Key Trends Shaping the Future of AI",
          "slides": [
            {{"slide_title": "Slide 1 Title", "slide_body": "Slide 1 Body"}},
            {{"slide_title": "Slide 2 Title", "slide_body": "Slide 2 Body"}},
            {{"slide_title": "Slide 3 Title", "slide_body": "Slide 3 Body"}},
            {{"slide_title": "Slide 4 Title", "slide_body": "Slide 4 Body"}},
            {{"slide_title": "Slide 5 Title", "slide_body": "Slide 5 Body"}}
          ],
          "hashtags": ["#AI", "#Tech", "#FutureofWork"]
        }}

        Generated Carousel Post:
        """
        prompt = PromptTemplate(
            input_variables=["name", "role", "company", "skills", "interests", "brand_voice", "trend", "language"],
            template=template,
        )
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        response = llm_chain.run(
            name=profile["name"],
            role=profile["role"],
            company=profile["company"],
            skills=", ".join(profile["skills"]),
            interests=", ".join(profile["interests"]),
            brand_voice=profile["brand_voice"],
            trend=trend,
            language=language
        )
        # Attempt to parse the JSON output from the model
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print("Failed to parse carousel JSON. Returning raw text.")
            return {"title": "Error generating carousel", "slides": [], "hashtags": []}

    # Placeholder for other post types like articles and polls
    return "Unsupported post type."

def generate_network_strategies(profile):
    """Generates a list of AI-powered network growth strategies."""
    template = """
    You are an expert LinkedIn growth strategist. Based on the user's profile, provide 5 actionable strategies to grow their professional network and increase engagement.
    The strategies should be specific and practical.

    User Profile:
    - Name: {name}
    - Role: {role} at {company}
    - Skills: {skills}
    - Interests: {interests}

    Instructions:
    1. Provide a title for each strategy.
    2. Write a short, clear description for each strategy.
    3. The strategies should focus on connecting, engaging with content, and personal branding.
    4. Format the output as a JSON array of objects.

    Example JSON output:
    [
      {{
        "title": "Strategy 1 Title",
        "description": "Strategy 1 Description"
      }},
      {{
        "title": "Strategy 2 Title",
        "description": "Strategy 2 Description"
      }}
    ]

    Generated Strategies:
    """
    prompt = PromptTemplate(
        input_variables=["name", "role", "company", "skills", "interests"],
        template=template,
    )
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run(
        name=profile["name"],
        role=profile["role"],
        company=profile["company"],
        skills=", ".join(profile["skills"]),
        interests=", ".join(profile["interests"])
    )
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        return [{"title": "Error generating strategies", "description": response}]

def check_compliance(post_content):
    """Simple check for compliance and ethics using keywords."""
    flagged_words = ["investment advice", "guaranteed returns", "unethical", "illegal"]
    if any(word in str(post_content).lower() for word in flagged_words):
        return False, "Content contains a flagged keyword for compliance."
    return True, "Content is compliant."

def analyze_post(post):
    """Provides sentiment analysis on a truncated post."""
    truncated_post = post[:450]
    sentiment_result = sentiment_analyzer(truncated_post)[0]
    return {
        "sentiment_label": sentiment_result['label'],
        "sentiment_score": f"{sentiment_result['score']:.2f}"
    }

def generate_image_for_post(post_content, stable_diffusion_pipe):
    """Generates a professional social media image for the post with fine-tuned prompts."""
    image_prompt = (
        f"A professional, high-quality social media graphic related to: '{post_content}'. "
        "The style should be clean, modern, and visually engaging. Use a color palette of professional blues, whites, and greys. "
        "Include abstract shapes and lines related to data science, machine learning, or a futuristic tech theme. "
        "Focus on clarity and professionalism."
    )
    negative_prompt = (
        "low quality, blurry, ugly, noisy, bad art, amateur, multiple subjects, text, letters, messy, "
        "poorly-drawn face, cross-eyed, extra limbs, deformed, poorly lit, dark, boring"
    )
    image = stable_diffusion_pipe(image_prompt, negative_prompt=negative_prompt, num_inference_steps=50).images[0]

    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def analyze_competitor_content(competitor_name):
    """Simulates competitor analysis by scraping recent articles about a competitor."""
    url = f"https://news.google.com/search?q={competitor_name}&hl=en-US&gl=US&ceid=US:en"
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        headlines = [h.get_text() for h in soup.find_all('a', class_='JtKR7b')]
        return headlines
    except Exception:
        return []

# --- New and improved scheduling function ---
def schedule_post(platforms, post_content, image_data, scheduled_time, post_id, post_type):
    """
    This is the core function for scheduling.
    It simulates a scheduling process and then attempts to post to APIs.
    """
    post_details = {
        "post_id": post_id,
        "post_content": post_content,
        "image_data": image_data,
        "platforms": platforms,
        "scheduled_time": scheduled_time.isoformat(),
        "status": "scheduled",
        "post_type": post_type
    }
    scheduled_posts_db.append(post_details)
    print(f"⏳ Post scheduled for: {scheduled_time.strftime('%Y-%m-%d %H:%M:%S')}")

    # --- API Integration Placeholder ---
    # THIS IS WHERE YOU ADD YOUR API CALLS FOR EACH PLATFORM
    # The code below is a simplified placeholder that will always "succeed"
    # and update the analytics database. Replace it with your actual API calls
    # when you have your credentials ready.

    performance_metrics_db[post_id] = {
        "likes": random.randint(10, 50),
        "comments": random.randint(2, 10),
        "shares": random.randint(0, 5),
        "reach": random.randint(100, 500)
    }

    # For a real calendar, this would be a cron job, but for this demo, we simulate immediately.
    # The actual API calls from previous iterations would go here.
    try:
        if "Twitter" in platforms:
             import tweepy
             print("Simulating post to Twitter...")
             # client = tweepy.Client(...)
             # response = client.create_tweet(text=post_content)
             # print(f"✅ Post sent to Twitter. Response: {response}")

        if "Facebook" in platforms:
             import facebook
             print("Simulating post to Facebook...")
             # graph = facebook.GraphAPI(...)
             # response = graph.put_object("me", "feed", message=post_content)
             # print(f"✅ Post sent to Facebook. Response: {response}")

    except Exception as e:
        print(f"❌ Error posting to a social media platform: {e}")


    return {"message": "Post successfully scheduled! (API calls executed/simulated)"}

def get_scheduled_posts():
    """Returns all posts that have been scheduled."""
    return scheduled_posts_db

def get_post_analytics(post_id):
    """Returns analytics for a specific post."""
    return performance_metrics_db.get(post_id, {})

In [ ]:
# ==============================================================================
# 4. Flask Backend and Frontend
# ==============================================================================
app = Flask(__name__)
CORS(app)

# HTML/CSS/JS for the frontend
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Influence OS - AI Intern Project</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif; max-width: 900px; margin: auto; padding: 20px; line-height: 1.6; background-color: #f0f2f5; color: #1c1e21; }
        h1, h2, h3 { color: #34495e; font-weight: 600; }
        .container { background: #ffffff; padding: 30px; border-radius: 12px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08); }
        .input-group { margin-bottom: 20px; }
        label { display: block; margin-bottom: 8px; font-weight: bold; color: #555; }
        input[type="text"], input[type="datetime-local"], select, textarea { width: 100%; padding: 12px; border: 1px solid #ccd0d5; border-radius: 6px; box-sizing: border-box; font-size: 16px; }
        button { background-color: #007bff; color: white; padding: 14px 24px; border: none; border-radius: 6px; cursor: pointer; font-size: 16px; font-weight: bold; margin-top: 15px; transition: background-color 0.3s ease; }
        button:hover { background-color: #0056b3; }
        .output-section { margin-top: 30px; padding: 25px; background-color: #e9ecef; border-radius: 8px; }
        .loading-spinner { border: 4px solid rgba(0, 0, 0, 0.1); width: 40px; height: 40px; border-radius: 50%; border-left-color: #007bff; animation: spin 1s ease infinite; margin: 25px auto; }
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
        pre { background-color: #f8f9fa; padding: 18px; border-radius: 6px; border: 1px solid #dee2e6; white-space: pre-wrap; word-wrap: break-word; font-size: 15px; line-height: 1.5; }
        .results-grid { display: grid; grid-template-columns: 1fr 1fr; gap: 20px; }
        .result-box { background-color: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05); }
        img { max-width: 100%; height: auto; border-radius: 6px; margin-top: 15px; border: 1px solid #ddd; }
        ul { padding-left: 20px; }
        .tab-button { background-color: #ddd; color: #333; padding: 10px 15px; border: none; cursor: pointer; transition: background-color 0.3s; }
        .tab-button.active { background-color: #007bff; color: white; }
        .tab-content { display: none; padding-top: 20px; }
        .tab-content.active { display: block; }
        #calendar-view {
            display: grid;
            grid-template-columns: repeat(7, 1fr);
            gap: 5px;
            background-color: #fff;
            padding: 10px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05);
        }
        .day-cell {
            min-height: 100px;
            border: 1px solid #eee;
            padding: 5px;
            border-radius: 4px;
            font-size: 12px;
            background-color: #f9f9f9;
        }
        .post-entry {
            background-color: #e6f7ff;
            border: 1px solid #91d5ff;
            border-radius: 3px;
            padding: 3px;
            margin-top: 3px;
            cursor: pointer;
            overflow: hidden;
            text-overflow: ellipsis;
            white-space: nowrap;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🚀 Influence OS - Social Media Automation</h1>
        <p>A full-stack, non-hardcoded AI project with interactive UI and advanced features.</p>

        <div>
            <button class="tab-button active" onclick="openTab(event, 'generator')">Content Generator</button>
            <button class="tab-button" onclick="openTab(event, 'calendar')">Content Calendar</button>
            <button class="tab-button" onclick="openTab(event, 'strategies')">Growth Strategies</button>
        </div>

        <div id="generator" class="tab-content active">
            <div class="input-form">
                <h2>User Profile</h2>
                <div class="input-group"><label>Name:</label><input type="text" id="name" value="Alex Johnson"></div>
                <div class="input-group"><label>Role:</label><input type="text" id="role" value="Data Scientist"></div>
                <div class="input-group"><label>Company:</label><input type="text" id="company" value="Tech Innovators Inc."></div>
                <div class="input-group"><label>Skills:</label><input type="text" id="skills" value="Machine Learning,Python,Data Visualization"></div>
                <div class="input-group"><label>Interests:</label><input type="text" id="interests" value="Generative AI,Ethical AI"></div>
                <div class="input-group"><label>Brand Voice:</label><input type="text" id="brand_voice" value="professional, informative"></div>

                <h2>Content & Research</h2>
                <div class="input-group"><label>Research Keyword:</label><input type="text" id="keyword" value="Generative AI"></div>
                <div class="input-group"><label>Post Type:</label>
                    <select id="post_type">
                        <option value="text">Text Post (A/B Test)</option>
                        <option value="carousel">Carousel Post</option>
                    </select>
                </div>
                <div class="input-group"><label>Language:</label><input type="text" id="language" value="English"></div>
                <div class="input-group"><label>Competitor Name:</label><input type="text" id="competitor" value="Google AI"></div>

                <button onclick="generateContent()">✨ Generate Content</button>
            </div>
            <div class="output-section" id="output">
                <h3>Output</h3>
                <p>Your results will appear here after generation.</p>
            </div>
        </div>

        <div id="calendar" class="tab-content">
            <h2>Content Calendar</h2>
            <div id="calendar-view"></div>
            <div id="post-analytics-details" class="output-section" style="margin-top: 20px;">
                <h3>Post Analytics</h3>
                <p>Click on a post in the calendar to see its analytics.</p>
            </div>
        </div>

        <div id="strategies" class="tab-content">
            <h2>AI-Powered Network Growth Strategies</h2>
            <div id="strategies-output" class="output-section">
                <p>Click the button below to generate strategies.</p>
            </div>
            <button onclick="generateStrategies()">💡 Generate Strategies</button>
        </div>

    </div>

    <script>
        const BASE_URL = window.location.origin;

        function openTab(evt, tabName) {
            var i, tabcontent, tablinks;
            tabcontent = document.getElementsByClassName("tab-content");
            for (i = 0; i < tabcontent.length; i++) {
                tabcontent[i].style.display = "none";
            }
            tablinks = document.getElementsByClassName("tab-button");
            for (i = 0; i < tablinks.length; i++) {
                tablinks[i].className = tablinks[i].className.replace(" active", "");
            }
            document.getElementById(tabName).style.display = "block";
            evt.currentTarget.className += " active";

            if (tabName === 'calendar') {
                renderCalendar();
            }
        }

        async function generateContent() {
            const userProfile = {
                name: document.getElementById('name').value,
                role: document.getElementById('role').value,
                company: document.getElementById('company').value,
                skills: document.getElementById('skills').value.split(','),
                interests: document.getElementById('interests').value.split(','),
                brand_voice: document.getElementById('brand_voice').value,
            };

            const data = {
                user_profile: userProfile,
                search_keyword: document.getElementById('keyword').value,
                language: document.getElementById('language').value,
                competitor_name: document.getElementById('competitor').value,
                post_type: document.getElementById('post_type').value,
            };

            const outputDiv = document.getElementById('output');
            outputDiv.innerHTML = '<h3>Generating Content...</h3><div class="loading-spinner"></div><p>This may take a few minutes as the models run.</p>';

            try {
                const response = await fetch(`${BASE_URL}/generate_content`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(data),
                });

                const result = await response.json();

                if (response.ok) {
                    let htmlContent;
                    if (result.post_type === "text") {
                         htmlContent = `
                            <h3>Generated Posts (A/B Test)</h3>
                            <div class="results-grid">
                                <div class="result-box">
                                    <h4>Post A</h4>
                                    <p><strong>Sentiment:</strong> ${result.post_A_analytics.sentiment_label} (confidence: ${result.post_A_analytics.sentiment_score})</p>
                                    <pre>${result.post_A}</pre>
                                </div>
                                <div class="result-box">
                                    <h4>Post B</h4>
                                    <p><strong>Sentiment:</strong> ${result.post_B_analytics.sentiment_label} (confidence: ${result.post_B_analytics.sentiment_score})</p>
                                    <pre>${result.post_B}</pre>
                                </div>
                            </div>
                            <h3>AI-Generated Image</h3>
                            <img src="data:image/png;base64,${result.image_data}" alt="Generated Image">
                            <h3>Competitor Analysis (Headlines)</h3>
                            <ul>
                                ${result.competitor_headlines.map(headline => `<li>${headline}</li>`).join('') || '<li>No competitor headlines found.</li>'}
                            </ul>
                            <div class="input-group" style="margin-top: 30px;"><label>Schedule Post for:</label><input type="datetime-local" id="scheduled_time"></div>
                            <button onclick="schedulePost('${encodeURIComponent(result.post_A)}', '${result.image_data}', 'text')">⏳ Schedule Post</button>
                        `;
                    } else if (result.post_type === "carousel") {
                        let carouselSlidesHtml = result.carousel_post.slides.map(slide => `
                            <li><strong>${slide.slide_title}</strong>: ${slide.slide_body}</li>
                        `).join('');

                        htmlContent = `
                            <h3>Generated Carousel Post</h3>
                            <h4>${result.carousel_post.title}</h4>
                            <ul>${carouselSlidesHtml}</ul>
                            <p><strong>Hashtags:</strong> ${result.carousel_post.hashtags.join(' ')}</p>
                            <h3>AI-Generated Image (Placeholder)</h3>
                            <p>For carousels, you would generate multiple images, one for each slide. We'll use a single placeholder image here.</p>
                            <img src="data:image/png;base64,${result.image_data}" alt="Generated Image">
                            <div class="input-group" style="margin-top: 30px;"><label>Schedule Post for:</label><input type="datetime-local" id="scheduled_time"></div>
                            <button onclick="schedulePost('${encodeURIComponent(JSON.stringify(result.carousel_post))}', '${result.image_data}', 'carousel')">⏳ Schedule Post</button>
                        `;
                    }

                    outputDiv.innerHTML = htmlContent;
                } else {
                    outputDiv.innerHTML = `<h3>Error</h3><p>${result.error}</p>`;
                }
            } catch (error) {
                outputDiv.innerHTML = `<h3>Error</h3><p>Could not connect to the server.</p>`;
            }
        }

        async function schedulePost(postContent, imageData, postType) {
            const scheduledTime = document.getElementById('scheduled_time').value;
            if (!scheduledTime) {
                alert("Please select a date and time to schedule the post.");
                return;
            }

            const data = {
                post_content: postContent,
                image_data: imageData,
                platforms: ['Twitter', 'Facebook'],
                scheduled_time: scheduledTime,
                post_type: postType
            };

            try {
                const response = await fetch(`${BASE_URL}/schedule_post`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(data),
                });

                const result = await response.json();
                if (response.ok) {
                    alert('✅ ' + result.message);
                    renderCalendar(); // Refresh the calendar after scheduling
                } else {
                    alert(`Scheduling failed: ${result.error}`);
                }
            } catch (error) {
                alert('Could not connect to the scheduling server.');
            }
        }

        async function generateStrategies() {
            const userProfile = {
                name: document.getElementById('name').value,
                role: document.getElementById('role').value,
                company: document.getElementById('company').value,
                skills: document.getElementById('skills').value.split(','),
                interests: document.getElementById('interests').value.split(','),
            };

            const strategiesOutputDiv = document.getElementById('strategies-output');
            strategiesOutputDiv.innerHTML = '<h3>Generating Strategies...</h3><div class="loading-spinner"></div>';

            try {
                const response = await fetch(`${BASE_URL}/generate_strategies`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ user_profile: userProfile }),
                });

                const result = await response.json();
                if (response.ok) {
                    let strategiesHtml = result.strategies.map(s => `
                        <h4>${s.title}</h4>
                        <p>${s.description}</p>
                    `).join('');
                    strategiesOutputDiv.innerHTML = `<h3>Generated Strategies</h3>${strategiesHtml}`;
                } else {
                    strategiesOutputDiv.innerHTML = `<h3>Error</h3><p>${result.error}</p>`;
                }
            } catch (error) {
                strategiesOutputDiv.innerHTML = `<h3>Error</h3><p>Could not connect to the server.</p>`;
            }
        }

        async function renderCalendar() {
            const calendarView = document.getElementById('calendar-view');
            const now = new Date();
            const year = now.getFullYear();
            const month = now.getMonth();
            const daysInMonth = new Date(year, month + 1, 0).getDate();

            calendarView.innerHTML = `
                <div class="day-cell"><strong>Sun</strong></div><div class="day-cell"><strong>Mon</strong></div><div class="day-cell"><strong>Tue</strong></div><div class="day-cell"><strong>Wed</strong></div><div class="day-cell"><strong>Thu</strong></div><div class="day-cell"><strong>Fri</strong></div><div class="day-cell"><strong>Sat</strong></div>
            `;

            const scheduledPosts = await getScheduledPosts();
            const postsByDay = {};
            scheduledPosts.forEach(post => {
                const date = new Date(post.scheduled_time);
                const day = date.getDate();
                if (date.getMonth() === month && date.getFullYear() === year) {
                  if (!postsByDay[day]) postsByDay[day] = [];
                  postsByDay[day].push(post);
                }
            });

            for (let i = 1; i <= daysInMonth; i++) {
                const dayCell = document.createElement('div');
                dayCell.className = 'day-cell';
                dayCell.innerHTML = `<strong>${i}</strong>`;
                if (postsByDay[i]) {
                    postsByDay[i].forEach(post => {
                        const postEntry = document.createElement('div');
                        postEntry.className = 'post-entry';
                        if (post.post_type === "text") {
                            postEntry.textContent = post.post_content.substring(0, 30) + '...';
                        } else if (post.post_type === "carousel") {
                            try {
                                const carouselContent = JSON.parse(decodeURIComponent(post.post_content));
                                postEntry.textContent = `Carousel: ${carouselContent.title.substring(0, 20)}...`;
                            } catch (e) {
                                postEntry.textContent = 'Carousel Post (parsing error)';
                            }
                        }
                        postEntry.onclick = () => showPostAnalytics(post.post_id);
                        dayCell.appendChild(postEntry);
                    });
                }
                calendarView.appendChild(dayCell);
            }
        }

        async function getScheduledPosts() {
            try {
                const response = await fetch(`${BASE_URL}/get_scheduled_posts`);
                if (response.ok) {
                    return await response.json();
                }
                return [];
            } catch (error) {
                return [];
            }
        }

        async function showPostAnalytics(postId) {
            const analyticsDiv = document.getElementById('post-analytics-details');
            analyticsDiv.innerHTML = `<h3>Fetching Analytics...</h3><div class="loading-spinner"></div>`;
            try {
                const response = await fetch(`${BASE_URL}/get_post_analytics?post_id=${postId}`);
                if (response.ok) {
                    const analytics = await response.json();
                    if (Object.keys(analytics).length > 0) {
                         analyticsDiv.innerHTML = `
                            <h3>Post Analytics (ID: ${postId})</h3>
                            <p><strong>Likes:</strong> ${analytics.likes}</p>
                            <p><strong>Comments:</strong> ${analytics.comments}</p>
                            <p><strong>Shares:</strong> ${analytics.shares}</p>
                            <p><strong>Reach:</strong> ${analytics.reach}</p>
                        `;
                    } else {
                        analyticsDiv.innerHTML = `<h3>Post Analytics</h3><p>No analytics found for this post.</p>`;
                    }
                } else {
                    analyticsDiv.innerHTML = `<h3>Post Analytics</h3><p>Could not fetch analytics.</p>`;
                }
            } catch (error) {
                analyticsDiv.innerHTML = `<h3>Post Analytics</h3><p>Error fetching data.</p>`;
            }
        }

        // Initialize with the first tab open
        document.addEventListener('DOMContentLoaded', () => {
            document.getElementById('generator').style.display = 'block';
            document.querySelector('.tab-button').className += ' active';
        });
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return html_content

@app.route('/generate_content', methods=['POST'])
def generate_content_endpoint():
    try:
        data = request.json
        user_profile = data['user_profile']
        search_keyword = data['search_keyword']
        language = data['language']
        competitor_name = data['competitor_name']
        post_type = data['post_type']

        print("\n--- Generating Content... ---")

        print(f"🔎 Researching Google News for '{search_keyword}' trends...")
        industry_trend = get_industry_trends(search_keyword)

        subreddit = find_subreddit(search_keyword)
        print(f"🔎 Researching Reddit for '{subreddit}' trends...")
        reddit_trend = get_reddit_trends(subreddit)

        combined_trend = f"{industry_trend}. Reddit: {reddit_trend}"

        print("\n✍️ Generating content...")
        if post_type == "text":
            generated_posts = generate_content(user_profile, combined_trend, "text", language=language)
            post_a = generated_posts[0].strip()
            post_b = generated_posts[1].strip() if len(generated_posts) > 1 else "Not available."

            # Compliance check
            is_compliant, compliance_reason = check_compliance(post_a)
            if not is_compliant:
                return jsonify({"error": f"Compliance check failed: {compliance_reason}"}), 400

            analysis_a = analyze_post(post_a)
            analysis_b = analyze_post(post_b) if post_b != "Not available." else None

            print("🎨 Generating AI-powered image...")
            image_data = generate_image_for_post(post_a, stable_diffusion_pipe)

            print("\n📈 Analyzing competitor...")
            competitor_headlines = analyze_competitor_content(competitor_name)

            response = {
                "post_A": post_a,
                "post_A_analytics": analysis_a,
                "post_B": post_b,
                "post_B_analytics": analysis_b,
                "image_data": image_data,
                "competitor_headlines": competitor_headlines,
                "post_type": "text"
            }
        elif post_type == "carousel":
            carousel_post = generate_content(user_profile, combined_trend, "carousel", language=language)

            # Compliance check for carousel content
            is_compliant, compliance_reason = check_compliance(json.dumps(carousel_post))
            if not is_compliant:
                return jsonify({"error": f"Compliance check failed: {compliance_reason}"}), 400

            print("🎨 Generating AI-powered image for carousel...")
            image_data = generate_image_for_post(carousel_post["title"], stable_diffusion_pipe)

            response = {
                "carousel_post": carousel_post,
                "image_data": image_data,
                "post_type": "carousel"
            }

        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/generate_strategies', methods=['POST'])
def generate_strategies_endpoint():
    try:
        data = request.json
        user_profile = data['user_profile']
        strategies = generate_network_strategies(user_profile)
        return jsonify({"strategies": strategies})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/schedule_post', methods=['POST'])
def schedule_post_endpoint():
    try:
        data = request.json
        platforms_to_post = data['platforms']
        post_content = data['post_content']
        image_data = data['image_data']
        post_type = data['post_type']

        scheduled_time_str = data.get('scheduled_time')
        scheduled_time = datetime.fromisoformat(scheduled_time_str)

        # In a real app, you would create a unique ID for the post, for now we use a timestamp
        post_id = int(time.time())
        schedule_post(platforms_to_post, post_content, image_data, scheduled_time, post_id, post_type)

        return jsonify({"message": "Post successfully scheduled!"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/get_scheduled_posts')
def get_scheduled_posts_endpoint():
    return jsonify(get_scheduled_posts())

@app.route('/get_post_analytics')
def get_post_analytics_endpoint():
    post_id = request.args.get('post_id')
    if not post_id:
        return jsonify({"error": "Post ID is required."}), 400

    analytics = get_post_analytics(int(post_id))
    return jsonify(analytics)

In [ ]:
# ==============================================================================
# 5. Run the Server
# ==============================================================================
# Terminate any existing ngrok tunnels
ngrok.kill()

# Set up the ngrok tunnel
PORT = 5000
public_url = ngrok.connect(PORT)
print(f"✨ Your Flask App is live! Click the link below to open it:\n{public_url}")

# Start the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=PORT, use_reloader=False)